# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [97]:
problem = '''A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared. 
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. '''

## 2. Generate the mathematical model

In [139]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [140]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model

**Parameters:**

*  `s_COLI`: Units of 1.5L COLI produced per hectolitre of syrup (40 units/hL)
*  `s_PEPSA`: Units of 1.5L PEPSA produced per hectolitre of syrup (20 units/hL)
*  `c_syrup`: Cost of buying and processing one hectolitre of syrup ($150/hL)
*  `capacity`: Net production capacity of the S1 line (7100 units/hour)
*  `hours`: Daily working hours (8 hours/day)

**Decision Variables:**

*  `X1`: Units of 1.5L COLI produced daily
*  `X2`: Units of 1.5L PEPSA produced daily
*  `H`: Hectolitres of syrup used daily

**Objective Function:**

Maximize daily profit (in dollars):

```
Maximize Z = 49000*X1 - X1^2 + 30*X2 - 2*X2^2 - c_syrup*H 
```

**Constraints:**

1. **Syrup Usage:** The amount of syrup used should be enough to produce the planned units of COLI and PEPSA:
   ```
   H >= (X1 / s_COLI) + (X2 / s_PEPSA)
   ```
2. **Production Capacity:** Total production cannot exceed the S1 line's capacity:
   ```
   X1 + X2 <= capacity * hours
   ```
3. **PEPSA Production Target:** PEPSA production should be at least half of COLI production:
   ```
   X2 >= 0.5 * X1 
   ```
4. **Non-Negativity:** Production quantities and syrup usage cannot be negative:
   ```
   X1 >= 0
   X2 >= 0
   H >= 0
   ```

This model aims to determine the optimal daily production quantities of COLI (X1) and PEPSA (X2), as well as the required amount of syrup (H), that maximize the daily profit for the firm, while adhering to the production capacity, syrup availability, and the firm's commitment to producing at least half the amount of PEPSA compared to COLI. 


## 3. Generate the pyomo code

In [141]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [142]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Parameters (using sample data as provided)
model.s_COLI = Param(initialize=40)  # Units of 1.5L COLI produced per hL of syrup
model.s_PEPSA = Param(initialize=20) # Units of 1.5L PEPSA produced per hL of syrup
model.c_syrup = Param(initialize=150) # Cost of buying and processing one hL of syrup
model.capacity = Param(initialize=7100)  # Net production capacity of the S1 line
model.hours = Param(initialize=8)      # Daily working hours

# Decision Variables
model.X1 = Var(domain=NonNegativeReals)  # Units of 1.5L COLI produced daily
model.X2 = Var(domain=NonNegativeReals)  # Units of 1.5L PEPSA produced daily
model.H = Var(domain=NonNegativeReals)   # Hectolitres of syrup used daily

# Objective Function
model.profit = Objective(expr=49000*model.X1 - model.X1**2 + 30*model.X2 - 2*model.X2**2 - model.c_syrup*model.H, sense=maximize)

# Constraints
model.syrup_usage = Constraint(expr=model.H >= (model.X1 / model.s_COLI) + (model.X2 / model.s_PEPSA))
model.production_capacity = Constraint(expr=model.X1 + model.X2 <= model.capacity * model.hours)
model.pepsa_target = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('glpk')  # You can choose a different solver if available
solver.solve(model)

# Print the results
print(f"Optimal COLI production (X1): {value(model.X1)} units")
print(f"Optimal PEPSA production (X2): {value(model.X2)} units")
print(f"Optimal syrup usage (H): {value(model.H)} hectolitres")
print(f"Maximum daily profit: ${value(model.profit)}") 
```


## 4. Input problem data and try running the generated code

In [144]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Parameters (using sample data as provided)
model.s_COLI = Param(initialize=40)  # Units of 1.5L COLI produced per hL of syrup
model.s_PEPSA = Param(initialize=20) # Units of 1.5L PEPSA produced per hL of syrup
model.c_syrup = Param(initialize=150) # Cost of buying and processing one hL of syrup
model.capacity = Param(initialize=7100)  # Net production capacity of the S1 line
model.hours = Param(initialize=8)      # Daily working hours

# Decision Variables
model.X1 = Var(domain=NonNegativeReals)  # Units of 1.5L COLI produced daily
model.X2 = Var(domain=NonNegativeReals)  # Units of 1.5L PEPSA produced daily
model.H = Var(domain=NonNegativeReals)   # Hectolitres of syrup used daily

# Objective Function
model.profit = Objective(expr=49000*model.X1 - model.X1**2 + 30*model.X2 - 2*model.X2**2 - model.c_syrup*model.H, sense=maximize)

# Constraints
model.syrup_usage = Constraint(expr=model.H >= (model.X1 / model.s_COLI) + (model.X2 / model.s_PEPSA))
model.production_capacity = Constraint(expr=model.X1 + model.X2 <= model.capacity * model.hours)
model.pepsa_target = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('ipopt')  # You can choose a different solver if available
solver.solve(model)

# Print the results
print(f"Optimal COLI production (X1): {value(model.X1)} units")
print(f"Optimal PEPSA production (X2): {value(model.X2)} units")
print(f"Optimal syrup usage (H): {value(model.H)} hectolitres")
print(f"Maximum daily profit: ${value(model.profit)}") 

Optimal COLI production (X1): 16335.833333340031 units
Optimal PEPSA production (X2): 8167.916666660053 units
Optimal syrup usage (H): 816.7916666646894 hectolitres
Maximum daily profit: $400289176.04199225


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.

In [145]:
print(response.text)

## Mathematical Optimization Model

**Parameters:**

*  `s_COLI`: Units of 1.5L COLI produced per hectolitre of syrup (40 units/hL)
*  `s_PEPSA`: Units of 1.5L PEPSA produced per hectolitre of syrup (20 units/hL)
*  `c_syrup`: Cost of buying and processing one hectolitre of syrup ($150/hL)
*  `capacity`: Net production capacity of the S1 line (7100 units/hour)
*  `hours`: Daily working hours (8 hours/day)

**Decision Variables:**

*  `X1`: Units of 1.5L COLI produced daily
*  `X2`: Units of 1.5L PEPSA produced daily
*  `H`: Hectolitres of syrup used daily

**Objective Function:**

Maximize daily profit (in dollars):

```
Maximize Z = 49000*X1 - X1^2 + 30*X2 - 2*X2^2 - c_syrup*H 
```

**Constraints:**

1. **Syrup Usage:** The amount of syrup used should be enough to produce the planned units of COLI and PEPSA:
   ```
   H >= (X1 / s_COLI) + (X2 / s_PEPSA)
   ```
2. **Production Capacity:** Total production cannot exceed the S1 line's capacity:
   ```
   X1 + X2 <= capacity * hours
   ``

In [146]:
print(response2.text)

```python
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Parameters (using sample data as provided)
model.s_COLI = Param(initialize=40)  # Units of 1.5L COLI produced per hL of syrup
model.s_PEPSA = Param(initialize=20) # Units of 1.5L PEPSA produced per hL of syrup
model.c_syrup = Param(initialize=150) # Cost of buying and processing one hL of syrup
model.capacity = Param(initialize=7100)  # Net production capacity of the S1 line
model.hours = Param(initialize=8)      # Daily working hours

# Decision Variables
model.X1 = Var(domain=NonNegativeReals)  # Units of 1.5L COLI produced daily
model.X2 = Var(domain=NonNegativeReals)  # Units of 1.5L PEPSA produced daily
model.H = Var(domain=NonNegativeReals)   # Hectolitres of syrup used daily

# Objective Function
model.profit = Objective(expr=49000*model.X1 - model.X1**2 + 30*model.X2 - 2*model.X2**2 - model.c_syrup*model.H, sense=maximize)

# Constraints
model.syrup_usage = Constraint(expr=model.H >= (model.X1 / 